In [ ]:
import os
import time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import random
import string

import torch
import torch.nn as nn           
import torch.nn.functional as F 
import torch.optim as optim
import torchvision.transforms as transforms
import training_helpers

from plot_volume import plot_volume
from dataset_stuff_64 import *
from model_64_conv4_no_bias import *
import visualtorch
from grokfast import gradfilter_ma, gradfilter_ema

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

print(device)

cuda


In [3]:
mask = training_helpers.load_mask(1, device=device)

In [4]:
batch_size = 32
val_dataloader, val_dataset = create_ect_dataloaders('../dataset/ect_data_eval.csv', batch_size=batch_size, SNR=None, augment=False) # SNR=None for no noise, number for SNR in dB

In [ ]:
folder = "./test_scheduler/"
name = "no_scheduler"

model = ECT_conv_net_3d(8).to(device)
training_helpers.initialize_weights(model, 'kaiming')

criterion_MSE = nn.MSELoss()
optimizer_g1 = optim.Adam(model.parameters(), lr=0.001)

train_dataloader, ect_dataset = create_ect_dataloaders('../dataset/ect_data_train.csv', batch_size=32, SNR=None, augment=True) # SNR=None for no noise, number for SNR in dB

start_time = time.time()
model, training_loss_arr, valid_loss_arr, lr_arr = training_helpers.train_model(model, device, mask, 50, train_dataloader, val_dataloader, criterion_MSE, optimizer_g1, scheduler=None)
training_time = time.time() - start_time

np.savetxt(folder + "training_time" + name + ".txt", np.array([training_time]), fmt="%.6f", header="Training time (seconds)")

np.save(folder + 'training_' + name + ''.join(random.choices(string.ascii_letters + string.digits, k=5)) + '.npy', training_loss_arr)
np.save(folder + 'val_' + name + ''.join(random.choices(string.ascii_letters + string.digits, k=5)) + '.npy', valid_loss_arr)
np.save(folder + 'lr_' + name + ''.join(random.choices(string.ascii_letters + string.digits, k=5)) + '.npy', lr_arr)

torch.save(model, folder + '64conv4_' + name + ''.join(random.choices(string.ascii_letters + string.digits, k=5)) + '.pt')
